In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import sklearn.metrics as me
import matplotlib.pyplot as plt

input_dir = "filter"

def dataset_split(df):
    x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.5, random_state=0)
    x_test, x_valid, y_test, y_valid = train_test_split(x_test, y_test, test_size=0.5, random_state=0)
    return x_train, x_valid, x_test, y_train, y_valid, y_test

datasets = ['ExoVarFiltered', 'HumVarFiltered', 'predictSNPSelected', 'SwissVarSelected', 'VariBenchSelected']
dfs = {}

for root, _, files in os.walk(input_dir):
    for file_ in files:
        dataset = file_.split(".")[0]
        df = pd.read_csv(os.path.join(root, file_))
        dfs[dataset] = df

In [ ]:
count_df = pd.DataFrame([[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0]], index=["Dataset", "Dataset-train", "Dataset-vaild" ,"Dataset-test","ExtraDataset"], columns=["Total", "Deleterious", "Benign"])
count_df.iat[0, 0] = len(df["True Label"])
count_df.iat[4, 0] = len(extra_df["True Label"])
df_count = df["True Label"].value_counts().to_dict()
extra_df_count = extra_df["True Label"].value_counts().to_dict()
count_df.iat[0, 1] = df_count[0]
count_df.iat[0, 2] = df_count[1]
count_df.iat[4, 1] = extra_df_count[0]
count_df.iat[4, 2] = extra_df_count[1]
count_df.iat[1, 0] = len(y_train)
count_df.iat[1, 1] = y_train.value_counts().to_dict()[0]
count_df.iat[1, 2] = y_train.value_counts().to_dict()[1]
count_df.iat[2, 0] = len(y_valid)
count_df.iat[2, 1] = y_valid.value_counts().to_dict()[0]
count_df.iat[2, 2] = y_valid.value_counts().to_dict()[1]
count_df.iat[3, 0] = len(y_test)
count_df.iat[3, 1] = y_test.value_counts().to_dict()[0]
count_df.iat[3, 2] = y_test.value_counts().to_dict()[1]
print(count_df)
import dataframe_image as dfi

df_styled = count_df.style.background_gradient()
dfi.export(df_styled, "count_df.png", fontsize=30)

In [ ]:
# 从5个数据集中抽出相同比例的训练、验证和测试集
x_train_merged = pd.DataFrame()
x_valid_merged = pd.DataFrame()
x_test_merged = pd.DataFrame()
y_train_merged = pd.Series()
y_valid_merged = pd.Series()
y_test_merged = pd.Series()
for dataset in ['ExoVarFiltered', 'HumVarFiltered', 'predictSNPSelected', 'VariBenchSelected']:
    df = dfs[dataset]
    x_train, x_valid, y_train, y_valid = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.4, random_state=0)
    x_valid, x_test, y_valid, y_test = train_test_split(x_valid, y_valid, test_size=0.5, random_state=0)
    x_train_merged = pd.concat([x_train_merged, x_train])
    x_valid_merged = pd.concat([x_valid_merged, x_valid])
    x_test_merged = pd.concat([x_test_merged, x_test])
    y_train_merged = pd.concat([y_train_merged, y_train])
    y_valid_merged = pd.concat([y_valid_merged, y_valid])
    y_test_merged = pd.concat([y_test_merged, y_test])

In [ ]:
# 使用gridSearchCV选择最优参数
from lightgbm.sklearn import LGBMClassifier
import numpy as np
from sklearn.model_selection import GridSearchCV

num_leaves = [ _ for _ in range(11, 60, 2)]
learning_rates = [0.1, 0.05, 0.01]
n_estimators = range(10, 150, 10)
max_depths = range(-1, 10, 1)
params_test = {
    "num_leaves" : num_leaves,
    "learning_rate": learning_rates,
    "n_estimators": n_estimators,
    "max_depth": max_depths
}

classifier = LGBMClassifier(objective="binary", metrics='auc', verbose=-1)
gsCV = GridSearchCV(classifier, params_test, verbose=4)
gsCV.fit(x_train, y_train, categorical_feature=["Chr", "Ref", "Alt", "A_Ref", "A_Alt"])

In [ ]:
params = {
    "num_leaves" : 31,
    "objective": "multiclass",
    "metric": 'multi_logloss',
    "max_bin": 50,
    "max_depth": 8,
    "learning_rate": 0.05,
    "is_unbalance": True,
    "num_class": 2
    # "silent": 1,  # 信息输出设置成1则没有信息输出
}

print("training begins")
bst = lgb.train(params, train_data, num_round, valid_sets=[valid_data, valid_data_2, valid_data_3], early_stopping_rounds=10) 
bst.save_model('model.txt', num_iteration=bst.best_iteration)
bst.save_model('mutationscore.model')
print("training finishes")


# 调参

## 准备数据

In [ ]:
# 合并后的数据集
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv("filter/merged_with_labels.csv")
features = ['Chr', 'Start', 'End', 'Ref', 'Alt', 'SIFT_score', 'Polyphen2_HDIV_score', 'Polyphen2_HVAR_score','LRT_score', 'MutationTaster_score', 'MutationAssessor_score',  'FATHMM_score', 'RadialSVM_score', 'VEST3_score', 'CADD_phred', 'A_Ref', 'A_Alt']

labels = ["SIFT_Label", "Polyphen2_HDIV_Label", "Polyphen2_HVAR_Label", "LRT_Label", "MutationTaster_Label", "MutationAssessor_Label", "FATHMM_Label", "RadialSVM_Label", "VEST3_score", "CADD_phred"]
# df = df[features]
 

x_train_all, x_valid, y_train, y_valid = train_test_split(df, df.iloc[:, 0], test_size=0.4, random_state=0)
x_valid_all, x_test_all, y_valid, y_test = train_test_split(x_valid, y_valid, test_size=0.5, random_state=0)

# only need some features as input of classifier
x_train = x_train_all[features]
x_valid = x_valid_all[features]
x_test = x_valid_all[features]

# remove those Nan from others classifier's results
x_test_nan_removed = x_test_all.dropna(subset=labels)

swissvar_test_all = pd.read_csv("filter/swissvar_with_labels.csv")
# swissvar_test_all = swissvar_test_all.dropna(subset=labels)
# swissvar_test_input = swissvar_test_all[features]

comparators = ['MCAP', 'REVEL', 'MVP_score','DANN_score', 'GERP++_RS']
swissvar_compare = swissvar_test_all.dropna(subset=comparators)
swissvar_input = swissvar_compare[features]
# swissvar_output = optimized_classifier.predict_proba(swissvar_input)
# swissvar_output = swissvar_output[:, 1]
swissvar_label = swissvar_compare["True Label"]

In [ ]:
import lightgbm as lgb

train_data = lgb.Dataset(x_train, y_train)
train_data.create_valid(swissvar_input, swissvar_label)

## n_estimators

In [ ]:
# 使用gridSearchCV选择最优参数
from lightgbm.sklearn import LGBMClassifier
import numpy as np
from sklearn.model_selection import GridSearchCV

# num_leaves = [ _ for _ in range(11, 60, 2)]
learning_rates = [0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3]
# n_estimators = range(10, 150, 10)
# max_depths = range(-1, 10, 1)
params = {
    # "num_leaves" : num_leaves,
    "learning_rate": learning_rates,
    # "n_estimators": n_estimators,
    # "max_depth": max_depths
    # "boosting" : ["gbdt", "rf", "dart", "goss"]
}

params = {    
          'boosting_type': 'gbdt',
          'objective': 'binary',
          'metric': 'auc',
          'learning_rate':0.1,
          'num_leaves':31, 
          'max_depth': 5,   
    }

cv_results = lgb.cv(params, train_data)
print(len(cv_results['auc-mean']))

# n_estimators= 100
# classifier = LGBMClassifier(objective="binary", metrics='auc', boosting="gbdt")
# gsCV = GridSearchCV(classifier, params, verbose=4)
# gsCV.fit(x_train, y_train, categorical_feature=["Chr", "Ref", "Alt", "A_Ref", "A_Alt"])

## max_depth 和num_leaves

In [ ]:
params = {
    "max_depth" : range(5, 10, 1),
    'num_leaves': range(110, 170, 10)
}

classifier = LGBMClassifier(objective="binary", metrics='auc', n_estimators=100, boosting="gbdt", learning_rate=0.1)
gsCV = GridSearchCV(classifier, params, verbose=4)
gsCV.fit(x_train, y_train, categorical_feature=["Chr", "Ref", "Alt", "A_Ref", "A_Alt"])

# max_depth=8 num_leaves=150


min_data_in_leaf 和 min_sum_hessian_in_leaf

In [ ]:
params = {
    'min_child_samples': [18, 19, 20, 21, 22],
    'min_child_weight':[0.001, 0.002]
}

classifier = LGBMClassifier(boosting_type="gbdt", num_leaves=150, max_depth=8, learning_rate=0.1, n_estimators=100, metrics='auc', bagging_fraction = 0.8, feature_fraction = 0.8)
gsCV = GridSearchCV(classifier, params, verbose=4)
gsCV.fit(x_train, y_train, categorical_feature=["Chr", "Ref", "Alt", "A_Ref", "A_Alt"])

# min_child_samples=20 min_child_weight=0.001

## bagging_fraction和feature_fraction

In [ ]:
params ={
    'feature_fraction': [0.5, 0.6, 0.7, 0.8, 0.9],
    'bagging_fraction': [0.6, 0.7, 0.8, 0.9, 1.0]
}

classifier = LGBMClassifier(boosting_type="gbdt", num_leaves=150, max_depth=8, learning_rate=0.1, n_estimators=100, metrics='auc', bagging_fraction = 0.8, feature_fraction = 0.8, min_child_samples=20, min_child_weight=0.001)
gsCV = GridSearchCV(classifier, params, verbose=4)
gsCV.fit(x_train, y_train, categorical_feature=["Chr", "Ref", "Alt", "A_Ref", "A_Alt"])

# bagging_fraction': 0.6, 'feature_fraction': 0.5

In [ ]:
params ={
    'reg_alpha': [0, 0.001, 0.01, 0.03, 0.08, 0.3, 0.5],
    'reg_lambda': [0, 0.001, 0.01, 0.03, 0.08, 0.3, 0.5]
}

classifier = LGBMClassifier(boosting_type="gbdt", num_leaves=150, max_depth=8, learning_rate=0.1, n_estimators=100, metrics='auc', min_child_samples=20, min_child_weight=0.001, bagging_fraction=0.6, feature_fraction=0.5)
gsCV = GridSearchCV(classifier, params, verbose=4)
gsCV.fit(x_train, y_train, categorical_feature=["Chr", "Ref", "Alt", "A_Ref", "A_Alt"])

# 'reg_alpha': 0.08, 'reg_lambda': 0.3

In [ ]:
print(gsCV.best_params_)
print(gsCV.best_score_)